In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import os 
import sqlite3
from credentials import key
import re
import numpy as np

In [2]:
# # api call for first half of results

# # set up the request parameters
# params_1 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'page': '1',
#     'max_page': '5'
# }

# # make the http GET request to RedCircle API
# api_result_1 = requests.get('https://api.redcircleapi.com/request', params_1)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_1.json()))
# json_data_1 = api_result_1.json()

In [3]:
# # api call for second half of results


# # set up the request parameters
# params_2 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'max_page': '8',
#     'page': '6'
# }


# # make the http GET request to RedCircle API
# api_result_2 = requests.get('https://api.redcircleapi.com/request', params_2)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_2.json()))
# json_data_2 = api_result_2.json()

In [4]:
# # save the 1st search results json in a .json file
# out_file = open("resources/search-results-1.json", "w")
  
# json.dump(json_data_1, out_file, indent = 4)
  
# out_file.close()

# # save the 2nd search results json in a .json file
# out_file = open("resources/search-results-2.json", "w")
  
# json.dump(json_data_2, out_file, indent = 4)
  
# out_file.close()

In [5]:
# read the 2 json search results as dicts

# read the first json files
f = open('resources/search-results-1.json')
# returns JSON object as a dictionary
search_data_1 = json.load(f)
# Closing file
f.close()

# read the first json files
f = open('resources/search-results-2.json')
# returns JSON object as a dictionary
search_data_2 = json.load(f)
# Closing file
f.close()

In [6]:
# create dataframe from 2 search results dicts
search_results_df_1 = pd.DataFrame(search_data_1['search_results'])
search_results_df_2 = pd.DataFrame(search_data_2['search_results'])
search_results_df = search_results_df_1.append(search_results_df_2, ignore_index=True)

search_results_df.head()

,position,product,offers,fulfillment,page,position_overall,seller
0,1,{'title': 'Gain + Aroma Boost Original Scent H...,"{'primary': {'price': 15.99, 'currency': 'USD'...",{'type': '1p'},1,1,NaN
1,2,{'title': 'Gain flings! Laundry Detergent Pacs...,"{'primary': {'price': 27.49, 'currency': 'USD'...",{'type': '1p'},1,2,NaN
2,3,{'title': 'Free Clear HE Liquid Laundry Deterg...,"{'primary': {'price': 7.69, 'currency': 'USD',...",{'type': '1p'},1,3,NaN
3,4,{'title': 'Arm &#38; Hammer Plus OxiClean Fre...,"{'primary': {'price': 14.69, 'currency': 'USD'...",{'type': '1p'},1,4,NaN
4,5,{'title': 'All Free Clear Liquid Concentrated ...,"{'primary': {'price': 13.29, 'currency': 'USD'...",{'type': '1p'},1,5,NaN


In [7]:
# Each column is a dictionary. Get the dictionary of product data from the 'product' column.
product_json = search_results_df.loc[8]['product']

# Each column is a dictionary. Get the dictionary of price data from the 'offers'column.
offers_json = search_results_df.loc[8]['offers']
# The price information is in the 'primary' key of the offers_json
price_json = offers_json['primary']
pprint(product_json)

{'brand': 'Everspring',
 'brand_link': 'https://www.target.com/b/everspring/-/N-gl7n0',
 'class_id': 2,
 'department_id': 3,
 'dpci': '003-02-6943',
 'feature_bullets': ['64 loads',
                     '2x concentrated',
                     'Removes the toughest stains',
                     '7 powerful stain-fighting enzymes',
                     '100% natural fragrance',
                     'Non-toxic to humans when used as directed',
                     'Not tested on animals',
                     '96% Biobased Product'],
 'images': ['https://target.scene7.com/is/image/Target/GUEST_66ac2473-06ad-44fe-8522-e8687e76c9ad',
            'https://target.scene7.com/is/image/Target/GUEST_921684f9-8b91-40d2-b0e8-123db90ed29d',
            'https://target.scene7.com/is/image/Target/GUEST_2c2b4b3d-8b9f-4972-9a33-73f4d60b0f37',
            'https://target.scene7.com/is/image/Target/GUEST_1123573a-34b3-427c-89b9-c6743e5d47c1'],
 'link': 'https://www.target.com/p/lavender-38-bergamot-liquid

In [8]:
# clean up the product dictionary for only what is needed
product_json = {
    'title': product_json['title'],
    'link': product_json['link'],
    'department_id': product_json['department_id'],
    'brand': product_json['brand'],
    'rating':product_json['rating'],
    'tcin': product_json['tcin'],
    'position': search_results_df.loc[8]['position']
}
product_json

{'title': 'Lavender &#38; Bergamot Liquid Laundry Detergent - 100 fl oz - Everspring&#8482;',
 'link': 'https://www.target.com/p/lavender-38-bergamot-liquid-laundry-detergent-100-fl-oz-everspring-8482/-/A-75663151',
 'department_id': 3,
 'brand': 'Everspring',
 'rating': 4.4,
 'tcin': '75663151',
 'position': 9}

In [9]:
# For each row of the search_results_df, make a dict for product_json and price_json and add it to the full_product_df.
price_x_product_df = pd.DataFrame(

                        {'title': pd.Series(dtype='str'),
                        'link': pd.Series(dtype='str'),
                        'department_id': pd.Series(dtype='int'),
                        'brand': pd.Series(dtype='str'),
                        'rating': pd.Series(dtype='float'),
                        'tcin': pd.Series(dtype='str'),
                        'price': pd.Series(dtype='float'),
                        'currency': pd.Series(dtype='str'),
                        'symbol': pd.Series(dtype='str'),
                        'sponsored': pd.Series(dtype='str')
                        }
                    )

for i in range(len(search_results_df)):

    product_json = search_results_df.loc[i]['product']
    price_json = search_results_df.loc[i]['offers']['primary']
    position = search_results_df.loc[i]['position']
    if 'rating' in product_json:
        r = product_json['rating']
    else:
        r = 0
    if 'sponsored' in product_json:
        s = product_json['sponsored']
    else:
        s = 'False'

    full_product_info = {
        'title': product_json['title'],
        'link': product_json['link'],
        'department_id': product_json['department_id'],
        'brand': product_json['brand'],
        'rating': r,
        'tcin': product_json['tcin'],
        'price': price_json['price'], 
        'currency': price_json['currency'], 
        'symbol': price_json['symbol'],
        'sponsored': s
    }

    price_x_product_df = price_x_product_df.append(full_product_info, ignore_index=True)

price_x_product_df


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored
0,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,True
1,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True
2,Free Clear HE Liquid Laundry Detergent - 100 f...,https://www.target.com/p/free-clear-he-liquid-...,3,up & up,0.0,13186521,7.69,USD,$,False
3,Arm &#38; Hammer Plus OxiClean Fresh Scent Li...,https://www.target.com/p/arm-38-hammer-plus-ox...,3,Arm & Hammer,0.0,47054194,14.69,USD,$,False
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False
...,...,...,...,...,...,...,...,...,...,...
194,Grab Green Garbage Disposal Freshener & Cleane...,https://www.target.com/p/grab-green-garbage-di...,3,Grab Green,0.0,88332246,5.95,USD,$,False
195,Grab Green Newborn Baby Laundry Detergent Pods...,https://www.target.com/p/grab-green-newborn-ba...,3,Grab Green,0.0,88267094,11.89,USD,$,False
196,"Grab Green 3 in 1 Laundry Detergent Pods, Pouc...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332226,12.99,USD,$,False
197,"Grab Green 3 in 1 Laundry Detergent Pods, Jumb...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332221,42.75,USD,$,False


In [10]:
# find duplicated titles
s = pd.Series(price_x_product_df.duplicated(subset=['title'], keep=False))
price_x_product_df2 = price_x_product_df.copy()
price_x_product_df2['duplicate_title'] = s
# print the duplicate title rows
price_x_product_df2 = price_x_product_df2.loc[price_x_product_df2['duplicate_title'] == True]
price_x_product_df2

,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,duplicate_title
0,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,True,True
1,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True,True
5,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,False,True
6,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,True,True
7,Gain Flings Super Fresh 3-in-1 with Febreze an...,https://www.target.com/p/gain-flings-super-fre...,3,Gain,0.0,84114502,21.49,USD,$,True,True
28,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True,True
29,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,True,True
34,Gain Flings Super Fresh 3-in-1 with Febreze an...,https://www.target.com/p/gain-flings-super-fre...,3,Gain,0.0,84114502,21.49,USD,$,True,True
55,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,False,True
64,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,False,True


In [11]:
price_x_product_df2.groupby(by=['title', 'sponsored']).count()

link  \
title                                              sponsored         
Gain + Aroma Boost Original Scent HE Compatible... True          1   
                                                   False         1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True          2   
                                                   False         1   
Gain flings! Laundry Detergent Pacs - Original ... True          2   
                                                   False         1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True          2   
                                                   False         1   

                                                              department_id  \
title                                              sponsored                  
Gain + Aroma Boost Original Scent HE Compatible... True                   1   
                                                   False                  1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True                   2   
                                                   False                  1   
Gain flings! Laundry Detergent Pacs - Original ... True                   2   
                                                   False                  1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True                   2   
                                                   False                  1   

                                                              brand  rating  \
title                                              sponsored                  
Gain + Aroma Boost Original Scent HE Compatible... True           1       1   
                                                   False          1       1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True           2       2   
                                                   False          1       1   
Gain flings! Laundry Detergent Pacs - Original ... True           2       2   
                                                   False          1       1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True           2       2   
                                                   False          1       1   

                                                              tcin  price  \
title                                              sponsored                
Gain + Aroma Boost Original Scent HE Compatible... True          1      1   
                                                   False         1      1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True          2      2   
                                                   False         1      1   
Gain flings! Laundry Detergent Pacs - Original ... True          2      2   
                                                   False         1      1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True          2      2   
                                                   False         1      1   

                                                              currency  \
title                                              sponsored             
Gain + Aroma Boost Original Scent HE Compatible... True              1   
                                                   False             1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True              2   
                                                   False             1   
Gain flings! Laundry Detergent Pacs - Original ... True              2   
                                                   False             1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True              2   
                                                   False             1   

                                                              symbol  \
title                                              sponsored           
Gain + Aroma Boost Original Scent HE Compatible... True            1   
                                                   

In [12]:
# all the sponsored rows are duplicates, so drop them
price_x_product_df = price_x_product_df.loc[price_x_product_df['sponsored'] != True]
price_x_product_df = price_x_product_df.sort_values('title')

ounces

In [13]:
temp =[]
primary_title = []
ounces = []

# separate the title from the other details
for x in price_x_product_df['title']:
    i = x.split('-')
    temp.append(i)

#kep track of number of rows with no size.
no_size = 0

# put the title and the oz/fl oz in separate lists
for t in temp:

    primary_title.append(t[0])

    size = ''

    for x in t:

        if re.search('oz', x):
            size = x
    
    if size == '':
        no_size += 1

    ounces.append(size)
print('Rows with no ounces:')
print(no_size)
    
# add these lists to the search results df
price_x_product_df['primary_title'] = primary_title
price_x_product_df['ounces'] = ounces
price_x_product_df.head(15)


Rows with no ounces:
25


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
149,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.0,81260425,13.39,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,65 fl oz
139,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92 fl oz
146,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,81260462,13.39,USD,$,False,9 Elements Liquid Laundry Detergent,65 fl oz
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141 fl oz
43,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141 fl oz
125,All Free Clear Mega Pacs - 74oz/56ct,https://www.target.com/p/all-free-clear-mega-p...,3,All,4.8,84232057,21.99,USD,$,False,All Free Clear Mega Pacs,74oz/56ct
20,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,51227316,15.49,USD,$,False,All Liquid Laundry Detergent,184.5 fl oz
119,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.7,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,39ct/25.8oz
53,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.39,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,60ct/24.7oz
87,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,75663993,12.99,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,56ct/39.5oz


In [14]:
# x = price_x_product_df.iloc[7:8]
# xounces = x['ounces']

# for p in xounces:

#     temp_size = p.split('/')
#     print('temp_size:', temp_size)

#     for t in temp_size:
#         print(t)
#         if t.find('oz') != -1:
#             print('oz yes')
#             list = t.split("oz")
#             print("list: ", list)
#             ounces = list[0]
#             print(ounces)

In [15]:
# # make size col have just the number of ounces
# temp_ounces = []

# for x in price_x_product_df['ounces']:
    
#     temp_size = x.split('/') # [65oz, 23ct]

#     for t in temp_size:
#         if t.find('oz') != -1:
#             list = t.split("oz")
#             ounces = list[0].split("fl")[0]


#     temp_ounces.append(ounces)

# for i in temp_ounces:
#     print(i)
# price_x_product_df['ounces'] = temp_ounces
# print(len(price_x_product_df))
# price_x_product_df.head(15)

In [16]:
# make size col have just the number of ounces
temp_ounces = []

for x in price_x_product_df['ounces']:

    size = ''
    
    temp_size = x.split('/') # [65oz, 23ct]

    for t in temp_size:
        if re.search('oz', t):
            size_list = re.findall("\d+", t)
            if len(size_list) <= 1:
                size = size_list[0]
            else:
                size = size_list[0] + '.' + size_list[1]


    temp_ounces.append(size)


price_x_product_df['ounces'] = temp_ounces
print(len(price_x_product_df))
price_x_product_df.head(15)

192


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
149,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.0,81260425,13.39,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,65
139,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
146,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,81260462,13.39,USD,$,False,9 Elements Liquid Laundry Detergent,65
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141
43,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141
125,All Free Clear Mega Pacs - 74oz/56ct,https://www.target.com/p/all-free-clear-mega-p...,3,All,4.8,84232057,21.99,USD,$,False,All Free Clear Mega Pacs,74
20,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,51227316,15.49,USD,$,False,All Liquid Laundry Detergent,184.5
119,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.7,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,25.8
53,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.39,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,24.7
87,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,75663993,12.99,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,39.5


In [17]:
# convert the blank ounces to nan and drop those rows
price_x_product_df['ounces'] = price_x_product_df['ounces'].replace('',np.NaN)
price_x_product_df = price_x_product_df.dropna()
print(len(price_x_product_df))
price_x_product_df.head()

167


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
149,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.0,81260425,13.39,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,65
139,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
146,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,81260462,13.39,USD,$,False,9 Elements Liquid Laundry Detergent,65
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141
43,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141


In [21]:
price_x_product_df.dtypes

title             object
link              object
department_id      int64
brand             object
rating           float64
tcin              object
price            float64
currency          object
symbol            object
sponsored         object
primary_title     object
ounces            object
dtype: object

In [18]:
# price_x_product_df['ounces'] = price_x_product_df['ounces'].astype(float)
# price_x_product_df.dtypes


In [19]:
# save the search results in csv
os.makedirs('resources', exist_ok=True)  
price_x_product_df.to_csv('resources/price-x-product-data.csv')  

In [20]:
# create sqlite db
# conn = sqlite3.connect(':memory:')

conn = sqlite3.connect('laundry_detergent.db')

c = conn.cursor()

conn.execute("DROP TABLE laundry_detergents")

# create able
c.execute("""CREATE TABLE laundry_detergents(
        title TEXT,
        link TEXT,
        department_id INTEGER,
        brand TEXT,
        rating FLOAT,
        tcin TEXT,
        price REAL,
        currency TEXT,
        symbol TEXT,
        sponsored INTEGER,
        primary_title TEXT,
        ounces INTEGER
    )""")

# add the price-x-product-data.csv data to table
data = pd.read_csv('resources/price-x-product-data.csv')
data.to_sql('laundry_detergents', conn, if_exists='replace', index=False)
# _________________________
# queries
# _________________________

c.execute("""
    SELECT t.c, t.primary_title FROM
        (SELECT count(primary_title) as c, primary_title
        FROM laundry_detergents
        GROUP BY primary_title) as t
    WHERE t.c > 1
    """)

# c.execute("""
#     SELECT * FROM laundry_detergents
# """)

# _________________________
# end queries
# _________________________

c.execute
rows = c.fetchall()

conn.commit()

conn.close()

# check if any rows are not laundry detergent by seeing if the word in product title.
# for row in rows:
#     if 'Detergent' not in row[0]:
#         print(row[0])

for row in rows:
    print(row)

(2, '9 Elements Lemon Scent Liquid Laundry Detergent ')
(2, 'All Mighty Pacs Free Clear Laundry Detergent Pacs ')
(2, 'Free + Clear &#38; Gentle Laundry Detergent ')
(2, 'Grove Co. Laundry Detergent Sheets ')
(2, 'Liquid Laundry Detergent Spring Breeze ')
(2, 'Persil Original Liquid Concentrated Laundry Detergent ')
(2, 'Seventh Generation Liquid Laundry Detergent ')
(2, 'Seventh Generation Liquid Laundry Detergent Soap ')
(2, 'Tide High Efficiency Liquid Laundry Detergent ')
(2, 'Tide Plus Febreze High Efficiency Liquid Laundry Detergent ')
(4, 'Tide Pods Laundry Detergent Pacs ')
(2, 'Ultra')


/Users/annaweeks/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
